Import libraries and load the data

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Import necessary libraries at the beginning
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Load the data
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_Phase_1/train.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_Phase_1/valid.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_Phase_1/test.csv')

Drop NaN rows and prepare the *data*

In [34]:
# Drop rows with missing label_2
train_data = train_data.dropna(subset=['label_2'])
valid_data = valid_data.dropna(subset=['label_2'])

# Define label columns
label_columns = ['label_1', 'label_2', 'label_3', 'label_4']
X_train = train_data.drop(label_columns, axis=1)
y_train = train_data['label_2']
X_valid = valid_data.drop(label_columns, axis=1)
y_valid = valid_data['label_2']

Standardize the data

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

Initial K-Nearest Neighbors model and Calculate initial accuracy

In [36]:
from sklearn.svm import SVC

model_initial = SVC(kernel='rbf', C=1.0, gamma='scale')
model_initial.fit(X_train_scaled, y_train)

y_pred_initial = model_initial.predict(X_valid_scaled)

accuracy_initial = accuracy_score(y_valid, y_pred_initial)
print("Initial Accuracy:", accuracy_initial)

Initial Accuracy: 0.9184782608695652


Feature selection using SelectKBest

In [37]:
from sklearn.feature_selection import SelectKBest, f_classif

k_best = SelectKBest(score_func=f_classif, k=600)
X_train_selected = k_best.fit_transform(X_train_scaled, y_train)
X_valid_selected = k_best.transform(X_valid_scaled)

K-Nearest Neighbors model after feature selection and Calculate accuracy

In [38]:
model_after_feature_selection = SVC(kernel='rbf', C=1.0, gamma='scale')
model_after_feature_selection.fit(X_train_selected, y_train)

y_pred_after_feature_selection = model_after_feature_selection.predict(X_valid_selected)

accuracy_after_feature_selection = accuracy_score(y_valid, y_pred_after_feature_selection)
print("Accuracy after Feature Selection:", accuracy_after_feature_selection)

Accuracy after Feature Selection: 0.9211956521739131


PCA for dimensionality reduction

In [39]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
pca.fit(X_train_selected)
X_train_pca = pca.transform(X_train_selected)
X_valid_pca = pca.transform(X_valid_selected)

K-Nearest Neighbors model after PCA and calcualte accuracy

In [ ]:
model_after_pca = SVC(kernel='rbf', C=1.0, gamma='scale')
model_after_pca.fit(X_train_pca, y_train)

y_pred_after_pca = model_after_pca.predict(X_valid_pca)

accuracy_after_pca = accuracy_score(y_valid, y_pred_after_pca)
print("Accuracy after PCA:", accuracy_after_pca)

Hyperparameter tuning using GridSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'degree': [2, 3],
}

random_search = RandomizedSearchCV(
    verbose=1,
    estimator=SVC(),
    param_distributions=param_dist,
    n_iter=2,  # Adjust the number of iterations
    scoring='accuracy',
    cv=5,  # Adjust the number of folds
    n_jobs=-1
)

random_search.fit(X_train_pca, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=2, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10], 'degree': [2, 3],
                                        'kernel': ['linear', 'rbf']},
                   scoring='accuracy', verbose=1)

In [ ]:
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

Calculate tuned accuracy

In [ ]:
y_pred_tuned = best_estimator.predict(X_valid_pca)

accuracy_tuned = accuracy_score(y_valid, y_pred_tuned)
print("Tuned Accuracy:", accuracy_tuned)

Tuned Accuracy: 0.7608695652173914


In [ ]:
best_params

{'kernel': 'linear', 'degree': 2, 'C': 0.1}

In [44]:

# Scale the test data using the same scaler that was fitted on the training data
df_test_X_scaled = scaler.transform(test_data)

# Select the same features using SelectKBest
df_test_X_selected = k_best.transform(df_test_X_scaled)

# Transform the test data using the same PCA model
pca_test_X = pca.transform(df_test_X_selected)

# Use the best-tuned model for predictions on the test data
y_pred_test = model_after_pca.predict(pca_test_X)

# Assuming 'y_pred_test' is a numpy array or a list
predictions_df = pd.DataFrame({'Predictions': y_pred_test})

predictions_df.to_csv('svm_predictions_label_2-Layer9.csv', index=False)